In [0]:
import random
import torch
import torch.nn as nn
import torch.optim as optim


In [0]:
new_lines_neu = []
categories_neu = {}
categories_neg = {}
categories_pos = {}

In [0]:

with open('BN_NEU.txt','r') as f:
  lines_neu = f.readlines()
  lines_neu = lines_neu[65:]
  for line in lines_neu:
      new_lines = line.split()
      categories_neu[new_lines[1]] = [0,0,1]


In [0]:
with open('BN_NEG.txt','r') as f:
  lines_neg = f.readlines()
  lines_neg = lines_neg[65:]
  for line in lines_neg:
      new_lines = line.split()
      categories_neg[new_lines[1]] = [0,1,0]

In [0]:
with open('BN_POS.txt','r') as f:
  lines_pos = f.readlines()
  lines_pos = lines_pos[65:]
  for line in lines_pos:
      new_lines = line.split()
      categories_pos[new_lines[1]] = [1,0,0]

In [0]:
#Here we merge the categories and convert them into a dictionary
merge = [categories_neg,categories_neu,categories_pos]
new_categories = {}
for x in merge:
  new_categories.update(x)
keys = list(new_categories.keys())
random.shuffle(keys)
new_categories1 = [(key, new_categories[key]) for key in keys]  

In [0]:
#Here we check the length of the categories dictionary
new_categories1 = dict(new_categories1)
print(len(new_categories1))

In [0]:
#Creating a new list to store the categories
categories = []
keys = list(new_categories1.keys())
for i,k in new_categories1.items():
  categories.append(k)

In [0]:
#Printing the length of categories list
print(len(categories))

In [0]:
#Storing all the letters in a set so that there is no redundancy
all_letters = set('কখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহড়ঢ়য়ৎ◌্◌ঁ◌ং◌ঃ◌ি◌া◌ী◌ু◌ূ◌ৃ◌ৄ◌ৢ◌ৣ ে◌ ৈ◌ ে◌া ে◌ৗaআiঈuঊঋeঐoঔ')
all_letters = list(all_letters)
print(len(all_letters))
n_letters = 65

In [0]:
import torch

def letterToIndex(letter):
  """This function maps letters to integers"""
  try:
    for i,k in enumerate(all_letters):
      if letter == k:
        return i
  except:
    print('please enter a valid character')

def letterToTensor(letter):
    """Here we convert a letter to a tensor"""
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1

    return tensor

def lineToTensor(line):
    """Here we convert lines to tensors"""
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        if letterToIndex(letter) != -1:
          tensor[li][0][letterToIndex(letter)] = 1
        
    return tensor

#Training with Conv1D,LSTM and Linear transformation layers.

In [0]:
class Conv_LSTM(nn.Module):
    """Here we build a custom class with Conv1D,LSTM and Linear transformation layers"""
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        
        super(Conv_LSTM, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.c1 = nn.Conv1d(input_size, hidden_size, 1)
        self.c2 = nn.Conv1d(hidden_size, hidden_size, 1)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=0.01)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, inputs, hidden):

        batch_size = inputs.size(1)
        inputs = inputs.transpose(0, 1).transpose(1, 2)
        c = self.c1(inputs)
        p = self.c2(c)
        p = p.transpose(1, 2).transpose(0, 1)        
        p = torch.tanh(p)
        output, hidden = self.lstm(p, hidden)
        conv_seq_len = output.size(0)
        output = output.view(conv_seq_len * batch_size, self.hidden_size)
        output = torch.tanh(self.out(output))
        output = output.view(conv_seq_len, -1, self.output_size)

        return output, hidden
#Declaration of Hyperparameters
input_size = n_letters
hidden_size = 50
output_size = 3
batch_size = 1
n_layers = 2
seq_len = 15
#Declaraing object of the model Conv_LSTM
rnn2 = Conv_LSTM(input_size, hidden_size, output_size, n_layers=n_layers)

In [0]:
def categoryFromOutput(output):
    """This function converts the output into one-hot vector"""
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    l = [0,0,0]
    l[category_i] = l[category_i]+1
    return l

In [0]:
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params = rnn2.parameters(),lr = 0.5) 

def train(category_tensors, line_tensors):
  """This function is used for training the model"""
  final_loss = 0
  correct_count = 0
  for i in range(0,100):
    line_tensor = line_tensors[i]
    category = category_tensors[i] 
    category_tensor = torch.tensor(category, dtype=torch.float)
    hidden = None
    
    for j in range(line_tensor.size()[0]):
        output,hidden = rnn2(line_tensor[j].view(1,-1).unsqueeze(1),hidden)
    guess_i = categoryFromOutput(output)
    
    if guess_i == category:
      correct =  '✓'
      correct_count = correct_count + 1
    else:
      correct =  '✗ (%s)' % category 
    loss = criterion(output.view(1,-1), category_tensor.view(1,3))
    final_loss = final_loss + loss

  final_loss = final_loss/100

  final_loss.backward()
  optimizer.step()

  return final_loss,correct_count  

In [0]:
n_iters = 3900
cat_tensors = []
line_tensors = []
all_losses = []
epochs = 5
#We train the model in batches and print out the accuracy 
for epoch in range(epochs):

  batch_num = 0
  correct_sum = 0
  final_loss = 0
  rnn2.zero_grad()
  optimizer.zero_grad()

  for iter in range(0,n_iters):
    
    while True:
      
      category = new_categories1[keys[iter]]
      cat_tensors.append(category)
      line_tensor = lineToTensor(keys[iter])
      line_tensors.append(line_tensor)
      if iter % 100 != 0 or iter == 0:
        break

      else:
        batch_num = batch_num + 1
        final_loss,correct = train(cat_tensors,line_tensors)
        correct_sum = correct_sum + correct
        cat_tensors.clear()
        line_tensors.clear()
        
        if iter != 0:
          print('%d %s %.4f  %s %s' % (batch_num, iter , final_loss, correct_sum,epoch))
          break

  print( 'accuracy is %.6f%%' % (correct_sum/3900 * 100)) 

In [0]:
def evaluate2(line_tensor):
    """This function is used to evaluate the model"""
    hidden = None
    for i in range(line_tensor.size()[0]):
      output,hidden = rnn2(line_tensor[i].view(1,-1).unsqueeze(1),hidden)
        
    return output

In [0]:
num_corr2 = 0
n_iters = 3900
#Here we evaluate the testing data
for i in range(n_iters+1,4902):
  
  category = new_categories1[keys[i]]
  category_tensor = torch.tensor(category, dtype=torch.float)
  line_tensor = lineToTensor(keys[i])
  output = evaluate2(line_tensor)
  guess_new = categoryFromOutput(output)  
  if guess_new == category:
    correct =  '✓'
    num_corr2 = num_corr2 + 1
  else:
    correct =  '✗ (%s)' % category 
  print('%d %d%%  %s' % (i, i / n_iters * 100,correct))

In [0]:
#Here we print the accuracy
print(num_corr2/10)

#Training with Conv1D,GRU and Linear transformation layers 

In [0]:
class Conv_GRU(nn.Module):
    """Here we build a custom class with Conv1D,GRU and Linear transformation layers"""
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(Conv_GRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.c1 = nn.Conv1d(input_size, hidden_size, 1)
        self.c2 = nn.Conv1d(hidden_size, hidden_size, 1)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=0.01)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, inputs, hidden):

        batch_size = inputs.size(1)
        inputs = inputs.transpose(0, 1).transpose(1, 2)
        c = self.c1(inputs)
        p = self.c2(c)
        p = p.transpose(1, 2).transpose(0, 1)
        p = torch.tanh(p)
        output, hidden = self.gru(p, hidden)
        conv_seq_len = output.size(0)
        output = output.view(conv_seq_len * batch_size, self.hidden_size) 
        output = torch.tanh(self.out(output))
        output = output.view(conv_seq_len, -1, self.output_size)

        return output, hidden

#Declaration of Hyperparameters
input_size = n_letters
hidden_size = 50
output_size = 3
batch_size = 1
n_layers = 2
seq_len = 15
#Declaraing object of the model Conv_GRU
rnn = Conv_GRU(input_size, hidden_size, output_size, n_layers=n_layers)

In [0]:
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params = rnn.parameters(),lr = 0.5)

def train(category_tensors, line_tensors):
  """This function is used for training the model"""
  final_loss = 0
  correct_count = 0
  for i in range(0,100):
    line_tensor = line_tensors[i]
    category = category_tensors[i] 
    category_tensor = torch.tensor(category, dtype=torch.float)
    hidden = None
    
    for j in range(line_tensor.size()[0]):
        output,hidden = rnn(line_tensor[j].view(1,-1).unsqueeze(1),hidden)
    guess_i = categoryFromOutput(output)
    
    if guess_i == category:
      correct =  '✓'
      correct_count = correct_count + 1
    else:
      correct =  '✗ (%s)' % category 
    loss = criterion(output.view(1,-1), category_tensor.view(1,3))
    final_loss = final_loss + loss

  final_loss = final_loss/100

  final_loss.backward()
  optimizer.step()

  return final_loss,correct_count  

In [0]:
n_iters = 3900
cat_tensors = []
line_tensors = []
all_losses = []
epochs = 5
#we train the model in batches and print out the accuracy 
for epoch in range(epochs):

  batch_num = 0
  correct_sum = 0
  final_loss = 0
  rnn.zero_grad()
  optimizer.zero_grad()

  for iter in range(0,n_iters):
    
    while True:
      
      category = new_categories1[keys[iter]]
      cat_tensors.append(category)
      line_tensor = lineToTensor(keys[iter])
      line_tensors.append(line_tensor)
      
      if iter % 100 != 0 or iter == 0:
        break

      else:
        batch_num = batch_num + 1
        final_loss,correct = train(cat_tensors,line_tensors)
        correct_sum = correct_sum + correct
        cat_tensors.clear()
        line_tensors.clear()

        if iter != 0:
          print('%d %s %.4f  %s %s' % (batch_num, iter , final_loss, correct_sum,epoch))
          break

  print( 'accuracy is %.6f%%' % (correct_sum/3900 * 100)) 

In [0]:
def evaluate(line_tensor):
    """This function is used to evaluate the model"""
    hidden = None
    for i in range(line_tensor.size()[0]):
      output,hidden = rnn(line_tensor[i].view(1,-1).unsqueeze(1),hidden)
        
    return output

In [0]:
num_corr = 0
n_iters = 3900
#Here we evaluate the testing data
for i in range(n_iters+1,4902):
  
  category = new_categories1[keys[i]]
  category_tensor = torch.tensor(category, dtype=torch.float)
  line_tensor = lineToTensor(keys[i])
  output = evaluate(line_tensor)
  guess_new = categoryFromOutput(output)

  if guess_new == category:

    correct =  '✓'
    num_corr = num_corr + 1

  else:

    correct =  '✗ (%s)' % category 
    
  print('%d %d%% %s' % (i, i / n_iters * 100,correct)) 

In [0]:
#Here we are the printing the accuracy of the testing data 
print(num_corr/10)

#Training with Conv1D and Linear Transformation Layers

In [0]:
class Convolution_Layer(nn.Module):
    """Here we build a custom class with Conv1D and linear transformation layers"""
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):

        super(Convolution_Layer, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.c1 = nn.Conv1d(in_channels, out_channels, kernel_size= 1)
        self.c2 = nn.Conv1d(out_channels,out_channels, kernel_size = 1)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, inputs, hidden):

        batch_size = inputs.size(1)
        c = self.c1(inputs)
        c = self.c2(c)
        output = c.view(-1,1)
        output = self.softmax(self.out(output))

        return output

    def initHidden(self):

        return torch.zeros(1, self.hidden_size)    

In [0]:
#Declaration of Hyperparameters
in_channels = n_letters
out_channels = 1
output_size = 3
batch_size = 1
n_layers = 2

#Declaraing object of the model Convolution_Layer
cnn = Convolution_Layer(in_channels, out_channels, output_size, n_layers=n_layers)

In [0]:
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adadelta(params = cnn.parameters(),lr = 0.5)

def train(category_tensors, line_tensors):
  """This function is used for training the model"""
  final_loss = 0
  correct_count = 0
  for i in range(0,100):
    line_tensor = line_tensors[i]
    category = category_tensors[i] 
    category_tensor = torch.tensor(category, dtype=torch.float)
    hidden = cnn.initHidden()
    
    for j in range(line_tensor.size()[0]):
        output = cnn(line_tensor[j].unsqueeze(2),hidden)
    guess_i = categoryFromOutput(output)
    
    if guess_i == category:
      correct =  '✓'
      correct_count = correct_count + 1
    else:
      correct =  '✗ (%s)' % category 
    loss = criterion(output, category_tensor.view(1,3))
    final_loss = final_loss + loss

  final_loss = final_loss/100

  final_loss.backward()
  optimizer.step()

  return final_loss,correct_count 

In [0]:

n_iters = 3900
cat_tensors = []
line_tensors = []
all_losses = []
epochs = 5
#We train the model in batches and print out the accuracy 
for epoch in range(epochs):

  batch_num = 0
  correct_sum = 0
  final_loss = 0
  cnn.zero_grad()
  optimizer.zero_grad()

  for iter in range(0,n_iters):
    
    while True:
      
      category = new_categories1[keys[iter]]
      cat_tensors.append(category)
      line_tensor = lineToTensor(keys[iter])
      line_tensors.append(line_tensor)

      if iter % 100 != 0 or iter == 0:
        
        break

      else:

        batch_num = batch_num + 1
        final_loss,correct = train(cat_tensors,line_tensors)
        correct_sum = correct_sum + correct
        cat_tensors.clear()
        line_tensors.clear()

        if iter != 0:
          print('%d %s %.4f  %s %s' % (batch_num, iter , final_loss, correct_sum,epoch))
          break

  print( 'accuracy is %.6f%%' % (correct_sum/3900 * 100)) 

In [0]:
def evaluate(line_tensor):
    """This function is used to evaluate the model"""
    hidden = cnn.initHidden()
    for i in range(line_tensor.size()[0]):
      output = cnn(line_tensor[i].unsqueeze(2),hidden) 
             
    return output

In [0]:
num_corr = 0
n_iters = 3900
#Here we evaluate the testing data
for i in range(n_iters+1,4902):
  
  category = new_categories1[keys[i]]
  category_tensor = torch.tensor(category, dtype=torch.float)
  line_tensor = lineToTensor(keys[i])
  output = evaluate(line_tensor)
  guess_new = categoryFromOutput(output)  
  if guess_new == category:

    correct =  '✓'
    num_corr = num_corr + 1

  else:

    correct =  '✗ (%s)' % category 
    
  print('%d %d%% %s' % (i, i / n_iters * 100,correct)) 

In [0]:
#Here we are priniting the accuracy of the testing data
print(num_corr/10)

#Training with LSTM and Linear transformation layers.

In [0]:
class SentimentNet(nn.Module):
    """Here we build a custom class with LSTM and Linear transformation layers"""
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):

        super(SentimentNet, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True,bidirectional = True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, hidden,batch_size):

        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.softmax(out)
        out = out.view(batch_size, -1)
        out = out[:,-3:]

        return out, hidden
    
    def init_hidden(self, batch_size):

        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers * 2, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers * 2, batch_size, self.hidden_dim).zero_().to(device))
        
        return hidden

In [0]:
#Declaration of hyperparameters
vocab_size = n_letters
output_size = 3
embedding_dim = n_letters
hidden_dim = 512
n_layers = 2
batch_size = 1

In [0]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [0]:
#Declaration of model object
model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.to(device)

lr=0.5
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adadelta(model.parameters(), lr=lr)

In [0]:
def categoryFromOutput(output):
    """This function converts the output into one-hot vector"""
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    l = [0,0,0]
    l[category_i] = l[category_i]+1
    return  l

In [0]:
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adadelta(params = cnn.parameters(),lr = 0.5)
def train(category_tensors, line_tensors):
  """This function is used for training the model"""
  final_loss = 0
  correct_count = 0
  for i in range(0,100):
    line_tensor = line_tensors[i]
    category = category_tensors[i] 
    category_tensor = torch.tensor(category, dtype=torch.float)
    hidden = model.init_hidden(batch_size)
    
    for j in range(line_tensor.size()[0]):
        output,hidden = model(line_tensor[j].to(device),hidden,batch_size)
    guess_i = categoryFromOutput(output)
    
    if guess_i == category:
      
      correct =  '✓'
      correct_count = correct_count + 1
    else:
      correct =  '✗ (%s)' % category 
    loss = criterion(output.to(device), category_tensor.view(1,3).to(device))
    final_loss = final_loss + loss

  final_loss = final_loss/100

  final_loss.backward()
  optimizer.step()

  return final_loss,correct_count  

In [0]:
n_iters = 3900
category_list = []
cat_tensors = []
line_tensors = []
all_losses = []
epochs = 2
#we train the model in batches and print out the accuracy 
for epoch in range(epochs):

  batch_num = 0
  correct_sum = 0
  final_loss = 0
  model.zero_grad()
  optimizer.zero_grad()

  for iter in range(0,n_iters):
    
    while True:
      
      category = new_categories1[keys[iter]]
      category_list.append(category)
      cat_tensors.append(category)
      line_tensor = lineToTensor(keys[iter])
      line_tensors.append(line_tensor)
      if iter % 100 != 0 or iter == 0:
        break
      else:
        batch_num = batch_num + 1
        final_loss,correct = train(cat_tensors,line_tensors)
        correct_sum = correct_sum + correct
        cat_tensors.clear()
        line_tensors.clear()
        if iter != 0:
          print('%d %s %.4f  %s %s' % (batch_num, iter , final_loss, correct_sum,epoch))
          break
  print( 'accuracy is %.6f%%' % (correct_sum/3900 * 100))     

 

In [0]:
def evaluate(line_tensor):
    """This function is used to evaluate the model"""
    hidden = model.init_hidden(batch_size)
    for i in range(line_tensor.size()[0]):
      output,hidden = model(line_tensor[i].to(device),hidden,batch_size)
        
    return output

In [0]:
num_corr = 0
n_iters = 3900
#Here we evaluate the testing data
for i in range(n_iters+1,4902):
  
  category = new_categories1[keys[i]]
  category_tensor = torch.tensor(category, dtype=torch.float)
  line_tensor = lineToTensor(keys[i])
  output = evaluate(line_tensor)
  guess_new = categoryFromOutput(output)  

  if guess_new == category:

    correct =  '✓'
    num_corr = num_corr + 1

  else:

    correct =  '✗ (%s)' % category 

  print('%d %d%% %s' % (i, i / n_iters * 100,correct))

In [0]:
#Here we are the printing the accuracy of the testing data 
print(num_corr/1000 * 100)

#Training with GRU and Linear transformation layers.

In [0]:
class SentimentNet2(nn.Module):
    """Here we build a custom class with GRU and Linear transformation layers"""
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):

        super(SentimentNet2, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True,bidirectional = True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, hidden,batch_size):

        x = x.long()
        embeds = self.embedding(x)
        gru_out, hidden = self.gru(embeds, hidden)
        gru_out = gru_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(gru_out)
        out = self.fc(out)
        out = self.softmax(out)
        out = out.view(batch_size, -1)
        out = out[:,-3:]

        return out, hidden
    
    def init_hidden(self, batch_size):
      
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers * 2, batch_size, self.hidden_dim).zero_().to(device)
                   
        return hidden

In [0]:
#Declaration of model object
model2 = SentimentNet2(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model2.to(device)
lr=0.5
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adadelta(model2.parameters(), lr=lr)

In [0]:

def train2(category_tensors, line_tensors):
  """This function is used for training the model"""
  final_loss = 0
  correct_count = 0
  for i in range(0,100):
    line_tensor = line_tensors[i]
    category = category_tensors[i] 
    category_tensor = torch.tensor(category, dtype=torch.float)
    hidden = model2.init_hidden(batch_size)
    
    for j in range(line_tensor.size()[0]):
        output,hidden = model2(line_tensor[j].to(device),hidden,batch_size)
    guess_i = categoryFromOutput(output)
    
    if guess_i == category:
      correct =  '✓'
      correct_count = correct_count + 1
    else:
      correct =  '✗ (%s)' % category 
    loss = criterion(output.to(device), category_tensor.view(1,3).to(device))
    final_loss = final_loss + loss

  final_loss = final_loss/100

  final_loss.backward()
  optimizer.step()

  return final_loss,correct_count  

In [0]:
n_iters = 3900
print_every = 1
cat_tensors = []
line_tensors = []
all_losses = []
epochs = 3
#We train the model in batches and print out the accuracy 
for epoch in range(epochs):

  batch_num = 0
  correct_sum = 0
  final_loss = 0
  model2.zero_grad()
  optimizer.zero_grad()

  for iter in range(0,n_iters):
    
    while True:
      
      category = new_categories1[keys[iter]]
      cat_tensors.append(category)
      line_tensor = lineToTensor(keys[iter])
      line_tensors.append(line_tensor)

      if iter % 100 != 0 or iter == 0:

        break

      else:
        batch_num = batch_num + 1
        final_loss,correct = train2(cat_tensors,line_tensors)
        correct_sum = correct_sum + correct
        cat_tensors.clear()
        line_tensors.clear()

        if iter != 0:
          
          print('%d %d%% %.4f  %s %s' % (batch_num, correct_sum/ iter * 100, final_loss, correct_sum,epoch))
          break

  print( 'accuracy is %.6f%%' % (correct_sum/3900 * 100))   

In [0]:
def evaluate2(line_tensor):
    """This function is used to evaluate the model"""
    hidden = model2.init_hidden(batch_size)
    for i in range(line_tensor.size()[0]):
      output, hidden = model2(line_tensor[i].to(device), hidden,batch_size)
       
    return output

In [0]:
num_corr2 = 0
n_iters = 3900
#Here we evaluate the testing data
for i in range(n_iters+1,4902):
  
  category = new_categories1[keys[i]]
  category_tensor = torch.tensor(category, dtype=torch.float)
  line_tensor = lineToTensor(keys[i])
  output = evaluate2(line_tensor)
  guess_new = categoryFromOutput(output)  
  if guess_new == category:
    correct =  '✓'
    num_corr2 = num_corr2+ 1
  else:
    correct =  '✗ (%s)' % category 
  print('%d %d%% %s' % (i, i / n_iters * 100,correct))

In [0]:
#Here we are the printing the accuracy of the testing data 
print(num_corr2/1000 * 100)